# Wine Quality Prediction

This notebook analyses the Red Wine Quality dataset and builds machine learning models to predict wine quality from 
<br>physicochemical properties such as acidity, alcohol and sulphates.

The workflow includes:
1. Exploratory Data Analysis (EDA)
2. Data preprocessing and feature engineering
3. Training and evaluating machine learning models
4. Drawing conclusion on feature importance and predictive performance

## 1. Import Libraries

We start by importing the required Python libraries for:
- **Data handling**: pandas, numpy
- **Visualisation**: matplotlib, seaborn
- **Machine learning**: scikit-learn (for models and evaluation)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score